In [1]:
import sys
sys.version

'3.6.7 (v3.6.7:6ec5cf24b7, Oct 20 2018, 03:02:14) \n[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)]'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import cv2

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.layers import Dropout,GaussianDropout
from keras.layers import GaussianNoise
from keras.layers import Activation
from keras.utils import to_categorical
from keras.models import load_model
from keras.constraints import max_norm
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
#from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import StandardScaler
%matplotlib inline

Using TensorFlow backend.


In [3]:
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten

In [4]:
df=pd.read_csv('/Users/anubhavjain/Desktop/sign-language-mnist/sign_mnist_train.csv')
d_test=pd.read_csv('/Users/anubhavjain/Desktop/sign-language-mnist/sign_mnist_test.csv')

In [5]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [6]:
df.loc[0:2,:]

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195


In [7]:
t=np.array(df.loc[0,:])

In [8]:
x_train=np.array(df[df.columns[df.columns!='label']])
x_label=np.array(df['label'])

In [9]:
y_test=np.array(d_test[d_test.columns[d_test.columns!='label']])
y_label=np.array(d_test['label'])

In [10]:
x_label=to_categorical(x_label)
y_label=to_categorical(y_label)

In [11]:
#x_train.shape
#np.unique(y_label)
no_classes=y_label.shape[1]

In [12]:
#x_train.shape
y_label.shape

(7172, 25)

In [13]:
datagen = ImageDataGenerator(rotation_range=20,width_shift_range=0.3,height_shift_range=0.15)

In [14]:
x_train=x_train.reshape(x_train.shape[0],28,28,1)
y_test=y_test.reshape(y_test.shape[0],28,28,1)

In [15]:
x_train = x_train.astype('float32')
y_test = y_test.astype('float32')

In [16]:
x_train=x_train/255
y_test=y_test/255

In [ ]:
print(x_train[0])

In [17]:
datagen.fit(x_train)

In [ ]:
#scalar = StandardScaler()

In [ ]:
#scalar.fit(x_train)

In [ ]:
#x_train = scalar.transform(x_train)
#y_test = scalar.transform(y_test)

In [ ]:
#x_train=x_train.reshape(x_train.shape[0],28,28,1)
#y_test=y_test.reshape(y_test.shape[0],28,28,1)

In [26]:
model=Sequential()

In [18]:
model=load_model('gesture_to_text.h5')

In [27]:
model.add(GaussianNoise(0.2,input_shape=(28,28,1)))
model.add(Conv2D(128,(5,5),strides=(1,1),activation='linear',input_shape=(28,28,1)))
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(MaxPooling2D((2,2),strides=(2,2)))

model.add(Conv2D(64,(2,2),strides=(1,1),activation='linear'))
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(MaxPooling2D((2,2),strides=(2,2)))

model.add(Conv2D(32,(2,2),strides=(1,1),activation='linear'))
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(MaxPooling2D((2,2),strides=(2,2)))
model.add(Flatten())

#,kernel_initializer='normal',kernel_constraint=max_norm(3))
model.add(Dropout(0.2))
model.add(Dense(300,activation='linear'))
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(GaussianDropout(0.2))
model.add(Dense(300,activation='linear'))
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(Dense(300,activation='linear'))
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(Dense(300,activation='linear'))
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(GaussianDropout(0.2))
model.add(Dense(300,activation='linear'))
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(Dense(300,activation='linear'))
#model.add(Activation('relu'))###
model.add(LeakyReLU(alpha=0.2))###
model.add(Dropout(0.2))
model.add(Dense(no_classes,activation='softmax'))

In [28]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,x_label,validation_data=(y_test,y_label),epochs=20,batch_size=19,shuffle=True,callbacks=[TensorBoard(log_dir='/tmp/gesture_to_text')])

In [ ]:
model.fit_generator(datagen.flow(x_train, x_label, batch_size=19),steps_per_epoch=len(x_train)/19,validation_data=(y_test,y_label),epochs=50,shuffle=True,callbacks=[TensorBoard(log_dir='/tmp/gesture_to_text2')])

In [21]:
model.evaluate(y_test,y_label)

7172/7172 [==============================] - 2s 244us/step


[0.30092615790029376, 0.9075571667596207]

In [ ]:
img = cv2.imread('/Users/anubhavjain/Desktop/sign-language-mnist/outpy1.png',0)

In [ ]:
pic_array = np.empty((28, 28))
for i in range(28):
    for j in range(28):
        #print(img.item(i,j))
        pic_array[i][j] = img.item(i,j)/255
# pic_array,pic_array.shape

In [ ]:
img.shape
pic_array = pic_array.reshape(1,784)

In [ ]:
pic_array = scalar.transform(pic_array)

In [ ]:
pic_array = pic_array.reshape(1,28,28,1)
pic_array.shape
#pic_array[0]

In [ ]:
t = model.predict(pic_array)
y = np.nonzero(t == max(t[0]))
t[0]

In [ ]:
y[1][0]

In [ ]:
chr(y[1][0] + 65)

In [ ]:
temp = x_train[1].reshape(28,28)
temp
#cv2.imwrite('/Users/anubhavjain/Desktop/sign-language-mnist/outpy2.png', temp)

In [ ]:
t_before=time.time()
scores=model.evaluate(y_test,y_label,verbose=0)
t_after=time.time()

In [ ]:
scores[1]

In [ ]:
(t_after-t_before)

In [24]:
model.save('gesture_to_text.h5')